# Project 4 - Real Estate Listing

### Install and import packages

In [1]:
!cat requirements.txt

# Existing Repository Requirements
langchain==0.0.305
openai==0.28.1
pydantic>=1.10.12
pytest>=7.4.0
sentence-transformers==3.4.1
transformers==4.49.0
chromadb==0.4.12
jupyter==1.0.0
tiktoken==0.4.0
pandas==2.2.3
lancedb==0.20.0


In [2]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 3.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 71.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 77.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.6/32.6 MB 42.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.1/468.1 kB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 73.9 MB/s eta 0:00:00:00:01


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.8/346.8 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 37.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 33.1 MB/s eta 0:00:0000:0100:01


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [5]:
pip install -U sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain.chat_models import ChatOpenAI # this is the new import statement
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, NonNegativeInt
from typing import List, Union
from sentence_transformers import SentenceTransformer
from langchain.prompts import ChatPromptTemplate
from lancedb.pydantic import vector, LanceModel
import lancedb

import pandas as pd
import numpy as np
import json

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "<YOUR-OPENAI-KEY"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

### 0 - Defining models

In [3]:
#Embedding Model
EMB_NAME = 'paraphrase-MiniLM-L6-v2'
emb_model = SentenceTransformer(EMB_NAME)

# LLM
model_name = 'gpt-3.5-turbo'
llm = OpenAI(model_name=model_name, temperature=0.5)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


### 1. Generating Real Estate Listings
We will generate housing listings based on the requirements for the project using `gpt-3.5-turbo` 

In [4]:
class HouseListings(BaseModel):
    Neighborhood: str = Field(description="The US neighborhood of the listing.")
    Price: NonNegativeInt = Field(description="The price of the listing in USD.")
    Bedrooms: NonNegativeInt = Field(description="The number of bedrooms of the listing.")
    Bathrooms: NonNegativeInt = Field(description="The number of bathrooms of the listing.")
    HouseSize: str = Field(description="The house size in square feet (sqft).")
    Description: str = Field(description="A short and brief description of the generated listing.")
    NeighborhoodDescription: str = Field(description="A short and brief description of the neighborhood of the generated listing.")
        
parser = PydanticOutputParser(pydantic_object=HouseListings)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"Neighborhood": {"title": "Neighborhood", "description": "The US neighborhood of the listing.", "type": "string"}, "Price": {"title": "Price", "description": "The price of the listing in USD.", "minimum": 0, "type": "integer"}, "Bedrooms": {"title": "Bedrooms", "description": "The number of bedrooms of the listing.", "minimum": 0, "type": "integer"}, "Bathrooms": {"title": "Bathrooms", "description": "The number of bathrooms of the listing.", "minimum": 0, "type": "integer"}, "HouseSize": {"title": "Housesize", "description": "

In [5]:
instruction = "Please generate at least 10 real estate listings in CSV format that conform to the provided schema."

# Create an instance of HouseListings
listing = HouseListings(
    Neighborhood="Green Oaks",
    Price=800000,
    Bedrooms=3,
    Bathrooms=2,
    HouseSize="2000 sqft",
    Description="Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.",
    NeighborhoodDescription="Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze."
)

# Convert the instance to a JSON string
sample = listing.json()

prompt = PromptTemplate(
    template="{instruction}\n Sample:{sample}\n{format_instructions}\n",
    input_variables=["instruction", "sample"],
    partial_variables={"format_instructions": parser.get_format_instructions},
)

query = prompt.format(
    instruction=instruction,
    sample=sample
)
print(query)

Please generate at least 10 real estate listings in CSV format that conform to the provided schema.
 Sample:{"Neighborhood": "Green Oaks", "Price": 800000, "Bedrooms": 3, "Bathrooms": 2, "HouseSize": "2000 sqft", "Description": "Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.", "NeighborhoodDescription": "Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the c

In [6]:
output = llm(query)
print(output)

[
    {"Neighborhood": "Downtown", "Price": 1200000, "Bedrooms": 4, "Bathrooms": 3, "HouseSize": "2500 sqft", "Description": "Luxurious penthouse in the heart of Downtown with stunning city views. This 4-bedroom, 3-bathroom home features high-end finishes, a gourmet kitchen, and a private rooftop terrace. Enjoy the convenience of urban living in this upscale residence.", "NeighborhoodDescription": "Downtown is a vibrant neighborhood with access to top-rated restaurants, shopping, and entertainment venues. Explore the nearby museums, theaters, and parks for endless entertainment options."},
    {"Neighborhood": "Suburban Heights", "Price": 650000, "Bedrooms": 5, "Bathrooms": 4, "HouseSize": "3000 sqft", "Description": "Spacious family home in the desirable Suburban Heights neighborhood. This 5-bedroom, 4-bathroom property offers a large backyard, updated kitchen, and a cozy fireplace. Perfect for families looking for a quiet and safe community.", "NeighborhoodDescription": "Suburban Hei

In [7]:
df = pd.DataFrame(json.loads(output))
df.head()

,Neighborhood,Price,Bedrooms,Bathrooms,HouseSize,Description,NeighborhoodDescription
0,Downtown,1200000,4,3,2500 sqft,Luxurious penthouse in the heart of Downtown w...,Downtown is a vibrant neighborhood with access...
1,Suburban Heights,650000,5,4,3000 sqft,Spacious family home in the desirable Suburban...,Suburban Heights is a family-friendly neighbor...
2,Waterfront Estates,1800000,6,5,4000 sqft,Elegant waterfront estate with panoramic views...,Waterfront Estates is an exclusive neighborhoo...
3,Mountain View,850000,3,2,2200 sqft,Charming cottage-style home with mountain view...,Mountain View is a serene neighborhood surroun...
4,Lakefront Living,1500000,4,3,2800 sqft,Modern lakefront home with stunning water view...,Lakefront Living offers a peaceful retreat for...


In [8]:
if not os.path.exists("./data"):
    os.makedirs("./data")
    
df.to_csv('./data/generated_listings.csv')

### Storing Listings in a Vector Database

* Embed vectors

In [9]:
listings_description = list(df["Description"].values)
listings_description[0]

'Luxurious penthouse in the heart of Downtown with stunning city views. This 4-bedroom, 3-bathroom home features high-end finishes, a gourmet kitchen, and a private rooftop terrace. Enjoy the convenience of urban living in this upscale residence.'

In [11]:
def generate_embeddings(emb_model, input_data: Union[str, list[str]]) -> np.ndarray:    
    embeddings = emb_model.encode(input_data)
    return embeddings

listings_embeddings = generate_embeddings(emb_model, listings_description)

In [12]:
listings_embeddings[0][:5]

array([ 0.6446955 ,  0.22669806, -0.06769277, -0.00564443, -0.26699436],
      dtype=float32)

In [13]:
df["vector"] = list(listings_embeddings)

In [14]:
df.head()

,Neighborhood,Price,Bedrooms,Bathrooms,HouseSize,Description,NeighborhoodDescription,vector
0,Downtown,1200000,4,3,2500 sqft,Luxurious penthouse in the heart of Downtown w...,Downtown is a vibrant neighborhood with access...,"[0.6446955, 0.22669806, -0.06769277, -0.005644..."
1,Suburban Heights,650000,5,4,3000 sqft,Spacious family home in the desirable Suburban...,Suburban Heights is a family-friendly neighbor...,"[0.2741348, -0.011072203, -0.13638352, 0.00962..."
2,Waterfront Estates,1800000,6,5,4000 sqft,Elegant waterfront estate with panoramic views...,Waterfront Estates is an exclusive neighborhoo...,"[0.6878431, 0.07589187, 0.33334884, 0.37532544..."
3,Mountain View,850000,3,2,2200 sqft,Charming cottage-style home with mountain view...,Mountain View is a serene neighborhood surroun...,"[0.17909326, 0.24359195, 0.14489402, 0.4584947..."
4,Lakefront Living,1500000,4,3,2800 sqft,Modern lakefront home with stunning water view...,Lakefront Living offers a peaceful retreat for...,"[0.4604398, -0.10688394, 0.060537763, 0.343593..."


* Connecting to the database and creating the table with embeddings

In [15]:
class RealEstateListings(LanceModel):
    Neighborhood: str
    Price: int
    Bedrooms: int
    Bathrooms: int
    HouseSize: str
    Description: str
    NeighborhoodDescription: str
    vector: vector(listings_embeddings.shape[1])

In [16]:
db = lancedb.connect("~/.lancedb")
table_name = "real_estate_listings"
db.drop_table(table_name, ignore_missing=True)
table = db.create_table(table_name, schema=RealEstateListings)

In [17]:
data_to_insert = df.to_dict(orient='records')

# Insert the data
table.add(data_to_insert)

In [18]:
table.head().to_pandas()

,Neighborhood,Price,Bedrooms,Bathrooms,HouseSize,Description,NeighborhoodDescription,vector
0,Downtown,1200000,4,3,2500 sqft,Luxurious penthouse in the heart of Downtown w...,Downtown is a vibrant neighborhood with access...,"[0.6446955, 0.22669806, -0.06769277, -0.005644..."
1,Suburban Heights,650000,5,4,3000 sqft,Spacious family home in the desirable Suburban...,Suburban Heights is a family-friendly neighbor...,"[0.2741348, -0.011072203, -0.13638352, 0.00962..."
2,Waterfront Estates,1800000,6,5,4000 sqft,Elegant waterfront estate with panoramic views...,Waterfront Estates is an exclusive neighborhoo...,"[0.6878431, 0.07589187, 0.33334884, 0.37532544..."
3,Mountain View,850000,3,2,2200 sqft,Charming cottage-style home with mountain view...,Mountain View is a serene neighborhood surroun...,"[0.17909326, 0.24359195, 0.14489402, 0.4584947..."
4,Lakefront Living,1500000,4,3,2800 sqft,Modern lakefront home with stunning water view...,Lakefront Living offers a peaceful retreat for...,"[0.4604398, -0.10688394, 0.060537763, 0.343593..."


## Building the User Preference Interface

In [24]:
SIMPLE_PROMPT =\
"""
Based on the <context> you will give a suggestion of a real estate listing depending on the <preferences>
---
<context>
{context}
</context>
---
<preferences>
{preferences}
</preferences>
"""

In [25]:
AUGMENTED_PROMPT =\
"""
You are an AI Assistant whose main goal is to recommend real estate listings based on the <preferences> of the customer. 
You will receive a description of the buyers preference that you can find under <preferences> tags and you will reply with an informal and descriptive tone based on the <context>, ensuring that the customer will want to buy the listing.
The descriptions should be unique, appealing, and tailored to the preferences provided.
Remember you cannot change any factual information.

<context>
{context}
</context>
---
<preferences>
{preferences}
</preferences>
"""

In [44]:
def app_response(llm, emb_model, table, buyer_preferences = None, augmented = False):
    
    #Initialize buyer_preferences
    if buyer_preferences is None:
        buyer_preferences = {}
        
    #Questions
    questions = [
        "How big do you want your house to be?",
        "What are 3 most important things for you in choosing this property?", 
        "Which amenities would you like?", 
        "Which transportation options are important to you?",
        "How urban do you want your neighborhood to be?"
    ]
    
    #Checking if all questions are there
    for question in questions:
        if question not in buyer_preferences:
            return {"next_question": question}
    
    #Unifying text
    preferences_text = "\n".join([f"- {q}: {buyer_preferences[q]}" for q in questions])
        
    #getting prompt if augmented
    prompt_template = ChatPromptTemplate.from_template(AUGMENTED_PROMPT if augmented else SIMPLE_PROMPT)

    
    #Connecting to the database
    db = lancedb.connect("~/.lancedb")  # Replace with your LanceDB path

    #Embeddings
    query_embedding = emb_model.encode(preferences_text)

    # Perform the similarity search -> top 3 results
    results = table.search(query_embedding).limit(3).to_list()

    # Display the results
    #for result in results:
    #    print(result["Description"])
    
    #getting the context
    context = "\n\n".join([f"Option {i+1}: {result['Description']}" for i, result in enumerate(results)])

    
    #Calling the prompt
    prompt = prompt_template.format(context=context, preferences=preferences_text)
    print(f"Generated Prompt:\n{prompt}")
    
    #Use LLM
    response_text = llm.predict(prompt)
    print("Response:")
    print(response_text)
    print("\n\n")

In [45]:
buyers_preferences = {
    "How big do you want your house to be?": "A three-bedroom house with a garden.",
    "What are 3 most important things for you in choosing this property?": "Quiet neighborhood, good schools, near shopping centers.",
    "Which amenities would you like?": "Garage, modern kitchen, smart home features.",
    "Which transportation options are important to you?": "Near a metro station, easy highway access, bike-friendly roads.",
    "How urban do you want your neighborhood to be?": "A mix between suburban calm and access to city life."
}

### First using the simple prompt

In [46]:
app_response(llm, emb_model, table, buyers_preferences, augmented = False)

Generated Prompt:
Human: 
Based on the <context> you will give a suggestion of a real estate listing depending on the <preferences>
---
<context>
Option 1: Spacious family home in the desirable Suburban Heights neighborhood. This 5-bedroom, 4-bathroom property offers a large backyard, updated kitchen, and a cozy fireplace. Perfect for families looking for a quiet and safe community.

Option 2: Luxurious penthouse in the heart of Downtown with stunning city views. This 4-bedroom, 3-bathroom home features high-end finishes, a gourmet kitchen, and a private rooftop terrace. Enjoy the convenience of urban living in this upscale residence.

Option 3: Charming cottage-style home with mountain views in the peaceful Mountain View neighborhood. This 3-bedroom, 2-bathroom property features a cozy fireplace, updated kitchen, and a spacious deck for outdoor entertaining. Perfect for nature lovers seeking a tranquil retreat.
</context>
---
<preferences>
- How big do you want your house to be?: A th

### Then the augmented prompt

In [47]:
app_response(llm, emb_model, table, buyers_preferences, augmented = True)

Generated Prompt:
Human: 
You are an AI Assistant whose main goal is to recommend real estate listings based on the <preferences> of the customer. 
You will receive a description of the buyers preference that you can find under <preferences> tags and you will reply with an informal and descriptive tone based on the <context>, ensuring that the customer will want to buy the listing.
The descriptions should be unique, appealing, and tailored to the preferences provided.
Remember you cannot change any factual information.

<context>
Option 1: Spacious family home in the desirable Suburban Heights neighborhood. This 5-bedroom, 4-bathroom property offers a large backyard, updated kitchen, and a cozy fireplace. Perfect for families looking for a quiet and safe community.

Option 2: Luxurious penthouse in the heart of Downtown with stunning city views. This 4-bedroom, 3-bathroom home features high-end finishes, a gourmet kitchen, and a private rooftop terrace. Enjoy the convenience of urban l